In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!pip install cohere


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.1 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import json
import PyPDF2
import textwrap

"""
This code iterates over a list of PDFs, extracts text, and divides it into chunks. Then it saves the chunks into a JSON file.
"""

pdf_dir = os.path.join(os.getcwd(), '/content/drive/MyDrive/one_arti_1')
json_file = os.path.join('/content/drive/MyDrive/kenz', 'data4.json')


pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

output = []

for pdf_file in pdf_files:
    with open(os.path.join(pdf_dir, pdf_file), "rb") as f:
        pdf_reader = PyPDF2.PdfReader(f)
        print(f'Reading pdf: {f}')
        num_pages = len(pdf_reader.pages)
        print(f'This pdf has {num_pages} pages')
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            content = page.extract_text()
            chunks = textwrap.wrap(content, 1000)

            for chunk in chunks:
                output.append({
                    "filename": pdf_file,
                    "page_number": page_num + 1,
                    "content": chunk
                })

with open(json_file, "w") as f:
    json.dump(output, f)

Reading pdf: <_io.BufferedReader name='/content/drive/MyDrive/one_arti_1/4.pdf'>
This pdf has 17 pages


In [ ]:
!pip install cohere

In [ ]:
import json
import re
import cohere
import time
import uuid

co = cohere.Client('GUMF9oMInIPQrGS1kZVtUtI488P5NOwBBiiwdiYJ')

# Load existing JSON data from the file
with open("/content/drive/MyDrive/kenz/data4.json", "r") as json_file:
    passages_data = json.load(json_file)

# Initialize the list to store the generated QA pairs
generated_qa_pairs = []

for entry in passages_data:
    # Keep generating until the answer_start is greater than 0
    while True:
        try:
            context = entry["content"]
            paper_name = entry["filename"]
            print(f'Trying this passage: {context[:100]}')

            response = co.generate(
                model='command',
                prompt=f'Generate five questions (not multiple-choice) related to the following passage. Each question must have its answer in the passage and be in the format:\n\n{{ "question": "", "answer": "" }}\n\nPassage: {context}\n\nQuestion:',
                max_tokens=300,
                temperature=0.7,
                k=0,
                stop_sequences=[],
                return_likelihoods='NONE'
            )

            generated_text = response.generations[0].text.strip()

            parts = re.split(r"Réponse:|Answer:|reponse:|answer:|Reponse:|réponse:", generated_text)
            if len(parts) == 2:
                generated_question = parts[0].strip()
                generated_answer = parts[1].strip()
            else:
                generated_question = generated_text
                generated_answer = ""



            answer_start = context.find(generated_answer)

            # Check the condition and regenerate if needed
            if answer_start == 0:

                print("Regenerating QA pair...")
                continue
            else:
                print(answer_start)
                break  # Exit the loop if the condition is satisfied

        except cohere.error.CohereAPIError:
            print('Cohere API error. Waiting for 65 seconds before retrying...')
            time.sleep(65)
            continue

    qa_pair = {
        "paragraphs": [
            {
                "context": context,
                "qas": [
                    {
                        "question": generated_question,
                        "id": str(uuid.uuid4()),
                        "answers": [
                            {
                                "text": generated_answer,
                                "answer_start": answer_start
                            }

                        ]
                    }
                ]
            }
        ]
    }
    print(qa_pair)

    generated_qa_pairs.append(qa_pair)

    print("QA pair generated.")

# Write the generated QA pairs to a JSON file
output_path = os.path.join('/content/drive/MyDrive/kenz', 'qa_dataset4.json')

with open(output_path, "w") as json_file:
    json.dump(generated_qa_pairs, json_file, indent=4)

print("All QA pairs generated and added to the JSON dataset.")


Trying this passage: Longformer: The Long-Document Transformer Iz BeltagyMatthew E. PetersArman Cohan Allen Institute 
-1
{'paragraphs': [{'context': 'Longformer: The Long-Document Transformer Iz Beltagy\x03Matthew E. Peters\x03Arman Cohan\x03 Allen Institute for Artiﬁcial Intelligence, Seattle, WA, USA fbeltagy,matthewp,armanc g@allenai.org Abstract Transformer-based models are unable to pro- cess long sequences due to their self-attention operation, which scales quadratically with the sequence length. To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer. Longformer’s attention mechanism is a drop-in replacement for the standard self-attention and combines a local windowed attention with a task moti- vated global attention. Following prior work on long-sequence transformers, we evaluate Longformer on character-level language mod- eling and a